In [9]:
import pandas as pd
import requests
import io
import os
import PyPDF2
import textract
import logging
from elasticsearch import Elasticsearch

In [2]:
logging.basicConfig(filename='lokalplan.log', filemode='w')

In [10]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# Create index

In [4]:
mapping = {
  "mappings": {
    "plan": { 
      "properties": { 
        "planid": { "type": "text"  }, 
        "dokument": { "type": "text"  }, 
      }
    }
  }
}

In [5]:
# ignore 400 cause by IndexAlreadyExistsException when creating an index
es.indices.create(index='plandata', ignore=400, body=mapping)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'plandata'}

# Add data

http://nitin-panwar.github.io/Elasticsearch-tutorial-for-beginners-using-Python/

In [13]:
df = pd.read_csv('data/lokalplan.csv')

In [14]:
df_test = df[df.komnr == 240]

https://medium.com/@rqaiserr/how-to-convert-pdfs-into-searchable-key-words-with-python-85aab86c544f

In [15]:
def download_pdf(folder, url):
    try:
        r = requests.get(url)
    except Exception as e:
        print(e)
        logging.warning(e)
    pdf_name = url.split('/')[-1]
    path = folder + pdf_name
    with open(path, 'wb') as file:
        file.write(r.content)
    return pdf_name

In [16]:
def get_document(url):
    try:
        r = requests.get(url)
    except Exception as e:
        print(e)
        logging.warning(e)
    pdf_file = io.BytesIO(r.content)
    pdfReader = PyPDF2.PdfFileReader(pdf_file)

    #discerning the number of pages will allow us to parse through all #the pages
    num_pages = pdfReader.numPages
    count = 0
    text = ""
    #The while loop will read each page
    while count < num_pages:
        try:
            pageObj = pdfReader.getPage(count)
            count +=1
            text += pageObj.extractText()
        except Exception as e:
            logging.warning(e)
            print(e, url)
            
    if text != "":
        text = text
    #If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text
    else:
        try:
            filename = download_pdf('./', url)
            text = textract.process('./'+filename, method='tesseract', language='dan')
            if os.path.exists(filename):
                os.remove(filename)
        except Exception as e:
            logging.warning(e)
            print(e, url)

    return text

In [ ]:
df_test.

In [17]:
df_test['dokument'] = df_test['doklink'].apply(get_document)

Unexpected escaped string: b'}' https://dokument.plandata.dk/20_1059555_APPROVED_1195548493922.pdf


Unexpected escaped string: b'}' https://dokument.plandata.dk/20_1059680_APPROVED_1195552020754.pdf


Unexpected escaped string: b'}' https://dokument.plandata.dk/20_1059734_APPROVED_1195553390922.pdf


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy [ipykernel_launcher.py:1]


In [18]:
df_test.to_csv('data/kommuner/plan.csv')

In [11]:
def df_to_es(row):
    
    data = plan = {
        "planid" : row['planid'],
        "dokument": row['dokument']
    }
    
    res = es.index(index='plandata',doc_type='plan', body=plan)

In [12]:
df2.apply(df_to_es, axis=1)

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
dtype: object

# Search data

In [183]:
def search_documents(index, word):
    query = {
      "query":{
            "wildcard": {
                "dokument": f"*{word}*"
            }
        }
    }
    
    res = es.search(index=index, body=query)
    
    return res

In [184]:
keywords = 'støjvold støjværn støjskærm'.split(' ')

In [189]:
search_documents('plandata', 'støj')

{'took': 13,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 3,
  'max_score': 1.0,
  'hits': [{'_index': 'plandata',
    '_type': 'plan',
    '_id': 'cGArZWYBpMqA1-ytBLu7',
    '_score': 1.0,
    '_source': {'planid': 3808416,
     'dokument': ' \n \nLOKALPLAN 163\n \nFor et erhvervsområde ved Slotsmarken\n \n \n \nLokalplan 163\n \n                                                                  \nHørsholm Kommune\n \n \n \n \n2\n \nHvad er en lokalplan?\n \nIfølge Lov \nom planlægning kan kommunalbestyrelsen beslutte at\n \nudarbejde en \nlokalplan, når en bestemt udvikling ønskes fremmet.\n \nKommunalbestyrelsen har endvidere pligt til\n \nat udarbejde en\n \nlokalplan, før der \ngennemføres større udstykninger eller større\n \nbygge\n-\n \neller anlægsarbejder, \nherunder nedrivning af bebyggelse.\n \nLokalplanen fastlægger bindende bestemmelser for ejere, lejere og\n \nbrugere af de \nejendomme, der er omfattet af l\n

# Delete data

In [186]:
#es.indices.delete(index='plandata', ignore=[400, 404])

# Sandkasse

In [1]:
import pdfminer

In [ ]:
with open('/tmp/metadata.pdf', 'wb') as f:
    f.write(response.content)